In [ ]:
import os

os.chdir("/mnt/openfact/users/msawinski/factue-task2")
from factue.methods.llm_langchain.llm import Llm

import json

In [2]:
chat_model = Llm(
    model_name="llama3.1:8b",  # type: ignore
    provider="ollama",  # type: ignore
    mode="chat",  # type: ignore
    temperature="0.0",  # type: ignore
)

In [ ]:
promt_format = """\nOutput ONLY JSON: in the format {{"rating": 0-10, "reason": "reason for rating","alternative": "formulate new one sentence claim that better represents the text of post}}."""

prmpt_system = """You are an expert journalist. Your task is to validate if a claim was accuratley extracted from a post. Compare it and determine how accurately the claim reflects post. You outout rating from 0 to 10, where 0 means the claim is completely different from original text and 10 means the claim most accurately reflects the post. If claim does not accurately reflect the post then generate alternative claim. Do not include additional information. Compare ONLY the claim text with post text. Do not infere information that is not directly included in post. Keep original wording when possible.Focus on the most important infrmation. The claim should be concise and reflect ONLY most important information from the post. Preserve names and named entities from the post."""


claim = """Photo shows Louis Armstrong as a child"""
post = """The Karnofsky Jewish family, who immigrated to the United States from Lithuania, employed a 7-year-old boy and adopted (so to speak) him into their home.  He was originally given homework to get food because he was a starving kid.  He remained under the Jewish families employ, until he was 12  Karnofsky gave him money to buy his first instrument, which was a common instrument in Jewish families.  They really admired his musical talent.Later, when he became a professional"""
prmpt_user = """POST:\n{post}\n\nCLAIM:\n{claim}"""

from langchain.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", prmpt_system + promt_format),
        ("human", prmpt_user),
    ]
)
prompt = prompt_template.invoke({"post": post, "claim": claim})
reposonse = chat_model.invoke(prompt).content
print(reposonse)

{
"rating": 0,
"reason": "The claim does not mention the Karnofsky family or their relationship with the boy, and only mentions Louis Armstrong's photo",
"alternative": "A young Jewish boy from Lithuania was employed by the Karnofsky family in the US"
}


In [ ]:
claim = """Photo shows Louis Armstrong as a child"""
while claim and len(claim) > 0:
    prompt = prompt_template.invoke({"post": post, "claim": claim})
    reposonse = chat_model.invoke(prompt).content

    try:
        response = json.loads(reposonse)
        rating = response["rating"]
        reason = response["reason"]
        alternative = response["alternative"]
        pp(str(rating) + claim + "|||" + reason)
    except:
        print("Error in response")
        print(reposonse)
        claim = None
    if claim != alternative and len(alternative) > 10 and alternative:
        claim = alternative

('0A young Jewish boy from Lithuania was employed by the Karnofsky family in '
 'the US|||The claim does not mention the Karnofsky family or their '
 "relationship with the boy, and only mentions Louis Armstrong's photo")
('8A young Jewish boy from Lithuania was employed and adopted by the Karnofsky '
 'family in the US.|||The claim accurately states that a young Jewish boy from '
 'Lithuania was employed by the Karnofsky family in the US, but it does not '
 'mention his age or the circumstances of his employment.')
('8A young Jewish boy from Lithuania was employed by and adopted into the '
 "Karnofsky family in the US.|||The claim accurately reflects the post's main "
 'information about the Karnofsky family employing and adopting a young Jewish '
 'boy from Lithuania, but it does not mention the age of the boy or his '
 'musical talent.')
('8A 7-year-old Jewish boy from Lithuania was employed by and eventually '
 'adopted into the Karnofsky family in the US.|||The claim accurately re

KeyboardInterrupt: 